# Testing power of agent architectures for graph isomorphism

## Setup

In [1]:
FORCE_CPU = False

SEED = 349287

DATASET_NAME = "er10000"
TEST_SIZE = 0.2

D_DECIDER = 16

BATCH_SIZE = 256
NUM_EPOCHS = 200
LEARNING_RATE = 0.01

In [2]:
from abc import ABC

import numpy as np

import torch
from torch import nn
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import random_split

from torch_geometric.data import Batch as GeometricBatch
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import BaseTransform

from jaxtyping import Float

from tqdm import tqdm

import plotly.graph_objs as go

from pvg.scenarios import GraphIsomorphismAgent
from pvg.data import GraphIsomorphismDataset, GraphIsomorphismData
from pvg.parameters import Parameters, GraphIsomorphismParameters

In [3]:
torch.manual_seed(SEED)
np.random.seed(SEED)

In [4]:
params = Parameters(
    scenario="graph_isomorphism",
    trainer="test",
    dataset=DATASET_NAME,
    max_message_rounds=1,
    graph_isomorphism=GraphIsomorphismParameters(
        prover_d_gnn=8,
        verifier_d_gnn=8,
    )
)

In [5]:
if not FORCE_CPU and torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


## Load dataset

In [6]:
class ScoreToBitTransform(BaseTransform):
    """A transform that converts the score to a bit indicating isomorphism."""
    def __call__(self, data):
        for store in data.node_stores:
            store.y = (store.wl_score == -1).long()
        return data

In [7]:
dataset = GraphIsomorphismDataset(params, transform=ScoreToBitTransform())
dataset

GraphIsomorphismDataset(name='er10000', num_features=1, num_pairs=10000)

In [8]:
train_dataset, test_dataset = random_split(dataset, (1 - TEST_SIZE, TEST_SIZE))

## Agents

In [9]:
class GraphIsomorphismSoloAgent(GraphIsomorphismAgent, ABC):
    def _build_model(self, num_layers: int, d_gnn: int, num_heads: int) -> nn.Module:
        # Build up the GNN module
        self.gnn, self.attention = self._build_gnn_and_attention(
            d_input=1,
            d_gnn=d_gnn,
            num_layers=num_layers,
            num_heads=num_heads,
        )

        # Build the decider, which decides whether the graphs are isomorphic
        self.decider = self._build_decider(
            d_gnn=d_gnn,
            d_decider=D_DECIDER,
            d_out=2,
        )

    def forward(
        self, data: GraphIsomorphismData | GeometricBatch
    ) -> Float[Tensor, "batch_size 2"]:
        # print("y", data.y)
        gnn_output, attention_output, _ = self._run_gnn_and_attention(data)
        gnn_attn_output = gnn_output + attention_output
        decider_logits = self.decider(gnn_output)
        return decider_logits
    
    def to(self, device: str | torch.device):
        self.gnn.to(device)
        self.attention.to(device)
        self.decider.to(device)
        return self


class GraphIsomorphismSoloProver(GraphIsomorphismSoloAgent):
    def __init__(self, params: Parameters, device: str | torch.device):
        super().__init__(params, device)
        self._build_model(
            num_layers=params.graph_isomorphism.prover_num_layers,
            d_gnn=params.graph_isomorphism.prover_d_gnn,
            num_heads=params.graph_isomorphism.prover_num_heads,
        )


class GraphIsomorphismSoloVerifier(GraphIsomorphismSoloAgent):
    def __init__(self, params: Parameters, device: str | torch.device):
        super().__init__(params, device)
        self._build_model(
            num_layers=params.graph_isomorphism.verifier_num_layers,
            d_gnn=params.graph_isomorphism.verifier_d_gnn,
            num_heads=params.graph_isomorphism.verifier_num_heads,
        )

In [10]:
prover = GraphIsomorphismSoloProver(params, device)
prover

GraphIsomorphismSoloProver(
  (gnn): Sequential(
    (0): Linear(1, 8, bias=True)
    (1): ReLU(inplace=True)
    (2): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=32, out_features=8, bias=True)
  ))
    (3): ReLU(inplace=True)
    (4): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=32, out_features=8, bias=True)
  ))
    (5): ReLU(inplace=True)
    (6): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=32, out_features=8, bias=True)
  ))
    (7): ReLU(inplace=True)
    (8): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=32, out_features=8, bias=True)
  ))
    (9): ReLU(inplace=True)
    (10): GINConv(nn=Sequential(
    (0): Line

In [11]:
verifier = GraphIsomorphismSoloVerifier(params, device)
verifier

GraphIsomorphismSoloVerifier(
  (gnn): Sequential(
    (0): Linear(1, 8, bias=True)
    (1): ReLU(inplace=True)
    (2): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=32, out_features=8, bias=True)
  ))
    (3): ReLU(inplace=True)
    (4): GINConv(nn=Sequential(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=32, out_features=8, bias=True)
  ))
  )
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
  )
  (decider): Sequential(
    (0): ReLU(inplace=True)
    (1): Linear(in_features=8, out_features=16, bias=True)
    (2): ReLU(inplace=True)
    (3): Reduce('pair batch_size max_nodes d_decider -> pair batch_size d_decider', 'max')
    (4): Rearrange('pair batch_size d_decider -> batch_size (pair d_decider)')
    (5): Linear(in_features=32, out_features=16, bi

## Train

In [12]:
for name, param in prover.named_parameters():
    if name.startswith("gnn") or name.startswith("attention"):
        param.requires_grad = False
for name, param in verifier.named_parameters():
    if name.startswith("gnn") or name.startswith("attention"):
        param.requires_grad = False

[autoreload of Sequential_944f97 failed: Traceback (most recent call last):
  File "/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/sam/.virtualenvs/pvg-experiments/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 168, in reload
    raise ModuleNotFoundError(f"spec not found for the module {name!r}", name=name)
ModuleNotFoundError: spec not found for the module 'Sequential_944f97'
]


In [13]:
optimizer_prover = torch.optim.Adam(prover.parameters(), lr=LEARNING_RATE)
optimizer_verifier = torch.optim.Adam(verifier.parameters(), lr=LEARNING_RATE)

test_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, follow_batch=["x_a", "x_b"]
)


def train_step(
    model: GraphIsomorphismSoloAgent, optimizer, data: GraphIsomorphismData
) -> tuple[float, float]:
    model.train()
    optimizer.zero_grad()
    pred = model(data)
    loss = F.cross_entropy(pred, data.y)
    loss.backward()
    optimizer.step()
    with torch.no_grad():
        accuracy = (pred.argmax(dim=1) == data.y).float().mean().item()
    return loss.item(), accuracy


prover.to(device)
verifier.to(device)


losses_prover = np.empty(NUM_EPOCHS)
accuracies_prover = np.empty(NUM_EPOCHS)
losses_verifier = np.empty(NUM_EPOCHS)
accuracies_verifier = np.empty(NUM_EPOCHS)
for epoch in range(NUM_EPOCHS):
    total_loss_prover = 0
    total_accuracy_prover = 0
    total_loss_verifier = 0
    total_accuracy_verifier = 0
    for data in tqdm(test_loader, desc=f"Epoch {epoch+1}"):
        data = data.to(device)
        loss, accuracy = train_step(prover, optimizer_prover, data)
        total_loss_prover += loss
        total_accuracy_prover += accuracy
        loss, accuracy = train_step(verifier, optimizer_verifier, data)
        total_loss_verifier += loss
        total_accuracy_verifier += accuracy
    losses_prover[epoch] = total_loss_prover / len(test_loader)
    accuracies_prover[epoch] = total_accuracy_prover / len(test_loader)
    losses_verifier[epoch] = total_loss_verifier / len(test_loader)
    accuracies_verifier[epoch] = total_accuracy_verifier / len(test_loader)
    print(
        f"Prover: loss: {losses_prover[epoch]:.4f}, "
        f"accuracy: {accuracies_prover[epoch]:.4%}"
    )
    print(
        f"Verifier: loss: {losses_verifier[epoch]:.4f}, "
        f"accuracy: {accuracies_verifier[epoch]:.4%}"
    )

Epoch 1: 100%|██████████| 32/32 [00:02<00:00, 13.35it/s]


Prover: loss: 0.6948, accuracy: 49.4629%
Verifier: loss: 0.6954, accuracy: 49.3652%


Epoch 2: 100%|██████████| 32/32 [00:00<00:00, 34.40it/s]


Prover: loss: 0.6934, accuracy: 50.6348%
Verifier: loss: 0.6935, accuracy: 50.8545%


Epoch 3: 100%|██████████| 32/32 [00:00<00:00, 41.18it/s]


...

Prover: loss: 0.6935, accuracy: 49.7437%
Verifier: loss: 0.6611, accuracy: 55.1636%


Epoch 199: 100%|██████████| 32/32 [00:00<00:00, 42.24it/s]


Prover: loss: 0.6934, accuracy: 49.8657%
Verifier: loss: 0.6610, accuracy: 55.1147%


Epoch 200: 100%|██████████| 32/32 [00:00<00:00, 42.41it/s]

Prover: loss: 0.6935, accuracy: 49.4751%
Verifier: loss: 0.6661, accuracy: 54.6387%


In [14]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.arange(NUM_EPOCHS), y=losses_prover, mode="lines", name="Prover Loss"
    )
)
fig.add_trace(
    go.Scatter(
        x=np.arange(NUM_EPOCHS), y=losses_verifier, mode="lines", name="Verifier Loss"
    )
)

fig.update_layout(
    title="Prover and Verifier Losses", xaxis_title="Epoch", yaxis_title="Loss"
)

fig.show()

In [15]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.arange(NUM_EPOCHS),
        y=accuracies_prover,
        mode="lines",
        name="Prover Accuracy",
    )
)
fig.add_trace(
    go.Scatter(
        x=np.arange(NUM_EPOCHS),
        y=accuracies_verifier,
        mode="lines",
        name="Verifier Accuracy",
    )
)

fig.update_layout(
    title="Prover and Verifier Accuracies", xaxis_title="Epoch", yaxis_title="Accuracy"
)

fig.show()

## Test

In [16]:
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False, follow_batch=["x_a", "x_b"]
)


def test_step(
    model: GraphIsomorphismSoloAgent, optimizer, data: GraphIsomorphismData
) -> tuple[float, float]:
    model.eval()
    with torch.no_grad():
        pred = model(data)
        loss = F.cross_entropy(pred, data.y)
        accuracy = (pred.argmax(dim=1) == data.y).float().mean().item()
    return loss.item(), accuracy


loss_prover = np.empty(NUM_EPOCHS)
accuracy_prover = np.empty(NUM_EPOCHS)
loss_verifier = np.empty(NUM_EPOCHS)
accuracy_verifier = np.empty(NUM_EPOCHS)

loss_prover = 0
accuracy_prover = 0
loss_verifier = 0
accuracy_verifier = 0
for data in tqdm(test_loader, desc=f"Testing..."):
    data = data.to(device)
    loss, accuracy = train_step(prover, optimizer_prover, data)
    loss_prover += loss
    accuracy_prover += accuracy
    loss, accuracy = train_step(verifier, optimizer_verifier, data)
    loss_verifier += loss
    accuracy_verifier += accuracy
loss_prover = loss_prover / len(test_loader)
accuracy_prover = accuracy_prover / len(test_loader)
loss_verifier = loss_verifier / len(test_loader)
accuracy_verifier = accuracy_verifier / len(test_loader)

print()
print("Final Results:")
print(
    f"Prover: loss: {losses_prover[epoch]:.4f}, "
    f"accuracy: {accuracies_prover[epoch]:.4%}"
)
print(
    f"Verifier: loss: {losses_verifier[epoch]:.4f}, "
    f"accuracy: {accuracies_verifier[epoch]:.4%}"
)

Testing...: 100%|██████████| 8/8 [00:00<00:00, 23.17it/s]


Final Results:
Prover: loss: 0.6935, accuracy: 49.4751%
Verifier: loss: 0.6661, accuracy: 54.6387%
